1. Convert pdf to image

In [ ]:
from pdf2image import convert_from_path
import os

In [4]:
#Xử lí PDF 1 thành hình ảnh dạng png từ trang 203 đến trang 395 (theo id)
pdf_file = "../data/pdf/PDF1.pdf"
output_folder = "../image/PDF1"


images = convert_from_path(
    pdf_file,
    first_page=203,
    last_page=395
)

for page_num, img in enumerate(images, start=203):
    filename = f"PDF1-{page_num}.png"
    save_path = os.path.join(output_folder, filename)
    img.save(save_path, "PNG")


In [5]:
#Xử lí PDF 3 thành hình ảnh dạng png từ trang 12 đến trang 118 (theo id)
pdf_file = "../data/pdf/PDF3.pdf"
output_folder = "../image/PDF3"

os.makedirs(output_folder, exist_ok=True)

start_page = 12
end_page = 118

images = convert_from_path(pdf_file, first_page=start_page, last_page=end_page)

for page_num, img in zip(range(start_page, end_page + 1), images):
    page_str = f"{page_num:03d}"           
    filename = f"PDF3-{page_str}.png"
    save_path = os.path.join(output_folder, filename)

    img.save(save_path, "PNG")